In [33]:
from pyscf import gto, scf, ao2mo, fci
import itertools
from copy import copy
import CC, Wick, GeneralisedWick
import numpy as np

In [2]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

In [3]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [4]:
h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

In [5]:
h1body = CC.get1bodyHamiltonian(mf)
h2body = CC.get2bodyHamiltonian(mf)

In [6]:
print(h1)
print(h1body)

[[-1.18985062e+00  2.60021255e-17]
 [-4.71423569e-17 -5.33749102e-01]]
-1.1898506186070186 * a^{0\alpha}a_{0\alpha}
 + 2.600212552842601e-17 * a^{0\alpha}a_{1\alpha}
 + -1.1898506186070186 * a^{0\beta}a_{0\beta}
 + 2.600212552842601e-17 * a^{0\beta}a_{1\beta}
 + -4.7142356915405136e-17 * a^{1\alpha}a_{0\alpha}
 + -0.5337491016607854 * a^{1\alpha}a_{1\alpha}
 + -4.7142356915405136e-17 * a^{1\beta}a_{0\beta}
 + -0.5337491016607854 * a^{1\beta}a_{1\beta}


In [7]:
print(0.5 * eri.reshape(2,2,2,2))
print(h2body)

[[[[3.27004756e-01 5.55111512e-17]
   [4.16333634e-17 3.22624713e-01]]

  [[5.89805982e-17 9.37609905e-02]
   [9.37609905e-02 5.55111512e-17]]]


 [[[5.20417043e-17 9.37609905e-02]
   [9.37609905e-02 2.77555756e-17]]

  [[3.22624713e-01 1.11022302e-16]
   [6.93889390e-17 3.39068092e-01]]]]
0.3270047556780619 * a^{0\alpha}a^{0\alpha}a_{0\alpha}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\alpha}a_{0\alpha}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\alpha}a_{1\alpha}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\alpha}a_{1\alpha}a_{1\alpha}
 + 0.3270047556780619 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{0\alpha}
 + 5.204170427930421e-17 * a^{0\alpha}a^{0\beta}a_{0\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{0\alpha}
 + 0.09376099045135429 * a^{0\alpha}a^{0\beta}a_{1\beta}a_{1\alpha}
 + 4.163336342344337e-17 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{0\alpha}
 + 0.3226247132942424 * a^{0\alpha}a^{1\alpha}a_{1\alpha}a_{0\alpha}
 + 2.77555756

In [8]:
h2bodyTensor = GeneralisedWick.Tensor("v", ['g','g'], ['g','g'])

In [55]:
#h2bodyTensor.array = 0.5 * eri.reshape(2,2,2,2).swapaxes(2,3).swapaxes(1,2)
h2bodyTensor.array = 2 * get2bodyHamiltonianArray(mf)

In [23]:
def getDiagrams(tensor, vacuum):
    Nocc = sum(vacuum)
    Norbs = len(vacuum)
    diagrams = []
    lowerGeneralIndexCount = sum(i == 'g' for i in tensor.lowerIndexTypes)
    lowerSplits = list(itertools.product(['h', 'p'], repeat=lowerGeneralIndexCount))
    upperGeneralIndexCount = sum(i == 'g' for i in tensor.upperIndexTypes)
    upperSplits = list(itertools.product(['h', 'p'], repeat=upperGeneralIndexCount))
    for lowerSplit in lowerSplits:
        lowerSlices = [slice(None)] * tensor.excitationRank
        lowerSplitIndexTypes = list(lowerSplit)
        lGI = 0
        newLowerIndexTypes = copy(tensor.lowerIndexTypes)
        for lI in range(len(newLowerIndexTypes)):
            if newLowerIndexTypes[lI] == 'g':
                newLI = lowerSplitIndexTypes[lGI]
                if newLI == 'h':
                    lowerSlices[lI] = slice(None,Nocc)
                elif newLI == 'p':
                    lowerSlices[lI] = slice(Nocc, None)
                newLowerIndexTypes[lI] = newLI
                lGI += 1
        for upperSplit in upperSplits:
#            print(lowerSplit, upperSplit)
            upperSlices = [slice(None)] * tensor.excitationRank
            upperSplitIndexTypes = list(upperSplit)
            uGI = 0
            newUpperIndexTypes = copy(tensor.upperIndexTypes)
            for uI in range(len(newUpperIndexTypes)):
                if newUpperIndexTypes[uI] == 'g':
                    newUI = upperSplitIndexTypes[uGI]
                    if newUI == 'h':
                        upperSlices[uI] = slice(None,Nocc)
                    elif newUI == 'p':
                        upperSlices[uI] = slice(Nocc, None)
                    newUpperIndexTypes[uI] = newUI
                    uGI += 1
            slices = tuple(lowerSlices + upperSlices)
#            print(lowerSplitIndexTypes)
#            print(upperSplitIndexTypes)
#            print(newLowerIndexTypes)
#            print(newUpperIndexTypes)
#            print(slices)
            diagram = GeneralisedWick.Tensor(tensor.name, newLowerIndexTypes, newUpperIndexTypes)
            diagram.array = tensor.array[slices]
            diagrams.append(diagram)
    return diagrams

In [56]:
h2Diagrams = getDiagrams(h2bodyTensor, [1,0])

In [57]:
print(h2Diagrams[0].getOperator(spinFree=True))

1.0 * a_{h_{3}\beta}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\beta}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}


In [28]:
print(h2Diagrams[0].array)

[[[[0.32700476]]]]


In [58]:
for diagram in h2Diagrams:
    print(diagram.array)
    print(diagram.getOperator(spinFree=True))

[[[[0.65400951]]]]
1.0 * a_{h_{3}\beta}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\beta}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}
[[[[8.32667268e-17]]]]
-1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\alpha}
 + -1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{0}\alpha}
[[[[1.04083409e-16]]]]
1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{0}\beta}
 + 1.0 * a_{h_{2}\beta}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\alpha}
 + 1.0 * a_{h_{2}\alpha}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\beta}
 + 1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{0}\alpha}
[[[[0.18752198]]]]
1.0 * a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{1}\beta}a_{p_{0}\beta}
 + 1.0 * a^{h_{0}\alpha}a^{h_{1}\beta}a_{p

In [53]:
def get2bodyHamiltonianArray(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2BodyArray = np.zeros((Norbs_,Norbs_,Norbs_,Norbs_))
    for p in range(Norbs_):
        for q in range(Norbs_):
            for r in range(p + 1):
                for s in range(q + 1):
                    x = int(p + Norbs_ * r - 0.5 * r * (r + 1))
                    y = int(q + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == r and q == s:
                        hamiltonian2BodyArray[p,q,r,s] += 0.5 * eri[x, y]
                    else:
                        hamiltonian2BodyArray[p,q,r,s] += 0.5 * eri[x, y]
                        hamiltonian2BodyArray[r,s,p,q] += 0.5 * np.conjugate(eri[x, y])
    return hamiltonian2BodyArray

In [54]:
print(2 * get2bodyHamiltonianArray(mf))

[[[[6.54009511e-01 8.32667268e-17]
   [1.04083409e-16 1.87521981e-01]]

  [[8.32667268e-17 6.45249427e-01]
   [0.00000000e+00 5.55111512e-17]]]


 [[[1.04083409e-16 0.00000000e+00]
   [6.45249427e-01 1.38777878e-16]]

  [[1.87521981e-01 5.55111512e-17]
   [1.38777878e-16 6.78136184e-01]]]]


In [50]:
print(eri.reshape(2,2,2,2).swapaxes(2,3).swapaxes(1,2))

[[[[6.54009511e-01 8.32667268e-17]
   [1.17961196e-16 1.87521981e-01]]

  [[1.11022302e-16 6.45249427e-01]
   [1.87521981e-01 1.11022302e-16]]]


 [[[1.04083409e-16 1.87521981e-01]
   [6.45249427e-01 1.38777878e-16]]

  [[1.87521981e-01 5.55111512e-17]
   [2.22044605e-16 6.78136184e-01]]]]
